In [1]:
import pandas as pd
import numpy as np
import requests
import re
from bs4 import BeautifulSoup

In [3]:
park_info = pd.read_csv('../data/park_info.csv')

In [4]:
park_code_ = park_info['parkCode'].unique()

# Convert values to uppercase and store them in a list
park_code_list = [value.upper() for value in park_code_]

In [5]:
len(park_code_list)

462

In [54]:
park_code = "TUAI"
url = f"https://irma.nps.gov/Stats/MvcReportViewer.aspx?_id=7e661565-c923-497d-8d4d-8fc0dcc43632&_m=Remote&_r=/NPS.Stats.Reports/Park Specific Reports/Recreation Visitors By Month (1979 - Last Calendar Year)&_15=True&_16=True&_18=True&_19=True&_34=False&_35=False&_39=880px&Park={park_code}"
page = requests.get(url)
if page.status_code == 200:
    try:
        soup = BeautifulSoup(page.content, "html.parser")
        #print(soup.prettify())
        target_div = soup.find("div")
        tables = target_div.find_all('table')

        result=[]
        for table in tables:
            # Extract data from each table
            rows = table.find_all('td')
            for column in rows:
                result.append(column)
        visitor_data=[]
        for item in result:
            visitor_data.append(item.text)
        start_index = 105
        stop_string = 'Recreation'
        visitor_data_select = []
        for index in range(start_index,len(visitor_data)):
            item = visitor_data[index]
    
            if stop_string in item:
                break
            visitor_data_select.append(item)
        visitor_data_select = visitor_data_select[:-1]
        num_columns = 14
        num_rows = len(visitor_data_select) // num_columns
        reshaped_data = pd.DataFrame(visitor_data_select).values.reshape(num_rows, num_columns)
        visitor_df = pd.DataFrame(reshaped_data[1:], columns=reshaped_data[0])
    except:
        print(f"The URL returned a response. Moving on to the next URL.")

In [55]:
visitor_df

,Year,JAN,FEB,MAR,APR,MAY,JUN,JUL,AUG,SEP,OCT,NOV,DEC,Total
0,2023,"1,086","2,314","2,842","2,456","2,499","2,341","2,079","1,088","1,178","1,368","1,758","1,073","22,082"
1,2022,495,888,"1,531","1,559","1,366","1,547","1,974","1,176","1,587","1,296","1,559","1,159","16,137"
2,2021,135,239,488,283,500,"1,096","1,792",889,562,739,874,460,"8,057"
3,2020,"1,575","3,358","1,327",0,0,0,0,25,40,227,181,110,"6,843"
4,2019,126,"2,391","3,801","3,351","3,181","3,195","3,610","1,814","2,073","2,171","3,217","1,167","30,097"
5,2018,"1,534","2,690","3,378","3,272","2,360","3,403","3,096","1,923","1,140","2,157","1,175",701,"26,829"
6,2017,"1,391","4,479","4,775","4,352","2,868","4,351","5,811","3,096","2,816","2,029","2,129","1,225","39,322"
7,2016,808,"2,297","2,400","2,488","2,385","3,200","5,221","2,444","2,630","3,007","2,583","1,670","31,133"
8,2015,"1,594","1,675","2,143","2,467","2,070","3,575","3,031","1,367","1,286","1,378","1,317",759,"22,662"
9,2014,786,"2,039","3,742","1,815","1,974","3,056","3,801","1,728",877,"2,186","1,820",495,"24,319"


In [49]:
start_index = 105
stop_string = 'Recreation'
selected_items = []
for index in range(start_index,len(visitor_data)):
    item = visitor_data[index]
    
    if stop_string in item:
        break
    selected_items.append(item)

In [77]:
visitor_dfs=[]
for codes in park_code_list:
    url = f"https://irma.nps.gov/Stats/MvcReportViewer.aspx?_id=7e661565-c923-497d-8d4d-8fc0dcc43632&_m=Remote&_r=/NPS.Stats.Reports/Park Specific Reports/Recreation Visitors By Month (1979 - Last Calendar Year)&_15=True&_16=True&_18=True&_19=True&_34=False&_35=False&_39=880px&Park={codes}"
    page = requests.get(url)
    if page.status_code == 200:
        try:
            soup = BeautifulSoup(page.content, "html.parser")
            target_div = soup.find("div")
            tables = target_div.find_all('table')

            result=[]
            for table in tables:
            # Extract data from each table
                rows = table.find_all('td')
                for column in rows:
                    result.append(column)
            
            visitor_data=[]
            for item in result:
                visitor_data.append(item.text)
            start_index = 119
            stop_string = 'Recreation'
            visitor_data_select = []
            for index in range(start_index,len(visitor_data)):
                item = visitor_data[index]
                if stop_string in item:
                    break
                visitor_data_select.append(item)
            visitor_data_select = visitor_data_select[:-1]
            num_columns = 14
            num_rows = len(visitor_data_select) // num_columns
            visitor_df = pd.DataFrame(visitor_data_select).values.reshape(num_rows, num_columns)
            visitor_df_2 = pd.DataFrame(visitor_df)
            visitor_df_2['parkCode'] = codes
            visitor_dfs.append(visitor_df_2)
        except Exception as e:
            continue


In [78]:
visitor_dfs[0]

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,parkCode


In [79]:
visitor_dfs[5]

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,parkCode
0,2023,"502,751","444,164","544,811","534,799","527,250","557,562","552,945","572,047","645,017",,,,"4,881,346",NATR
1,2022,"574,486","534,645","586,183","599,239","555,142","584,864","510,932","539,168","579,633","534,620","476,475","468,146","6,543,533",NATR
2,2021,"476,709","424,676","543,333","536,813","526,140","636,990","567,903","535,350","570,815","539,975","524,752","517,890","6,401,346",NATR
3,2020,"484,943","442,739","515,828","453,763","518,817","547,092","560,788","528,326","542,426","568,238","501,727","460,121","6,124,808",NATR
4,2019,"485,642","472,579","550,701","560,343","540,971","562,198","578,598","536,957","562,171","518,106","469,752","458,023","6,296,041",NATR
5,2018,"520,652","493,303","548,460","562,551","532,313","567,551","593,998","555,756","552,542","528,068","448,299","458,946","6,362,439",NATR
6,2017,"451,904","488,458","554,127","564,210","551,306","559,359","584,721","533,212","577,087","529,567","473,850","458,261","6,326,062",NATR
7,2016,"464,284","435,680","491,630","514,979","493,427","525,501","564,420","507,958","490,342","533,071","455,525","414,498","5,891,315",NATR
8,2015,"470,262","414,844","484,650","509,178","486,175","516,654","542,031","492,061","476,818","518,523","459,006","415,610","5,785,812",NATR
9,2014,"466,886","422,542","493,266","513,830","490,267","519,539","550,245","496,760","488,825","523,467","461,799","419,048","5,846,474",NATR


In [80]:
park_visitor_data = pd.concat(visitor_dfs[2:], ignore_index=True)

In [88]:
park_visitor_data.to_csv('../data/park_visitor_data.csv')

In [85]:
park_visitor_data.columns = ("YEAR", "JAN","FEB","MAR","APR","MAY","JUN","JUL","AUG","SEP","OCT","NOV","DEC","TOTAL","PARKCODE")